In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import *
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import copy

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_T = Policy()
records_reform = Records(puf)

policy_H = Policy()
records_org = Records(puf)
bhv = Behavior()

calcbase = Calculator(policy = policy_base, records = records_base)
calc_T = Calculator(policy = policy_T, records = records_reform)
calc_T_Be = Calculator(policy = policy_H, records = records_org,behavior = bhv)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform_T1 = {
2017: {'_II_rt1': [0.12],
'_II_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_II_rt2': [0.25],
'_II_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt3': [0.25],
'_II_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt4': [0.25],
'_II_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt5': [0.25],
'_II_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt6': [0.25],
'_II_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt7': [0.33],
'_PT_rt1': [0.12],
'_PT_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_PT_rt2': [0.25],
'_PT_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt3': [0.25],
'_PT_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt4': [0.25],
'_PT_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt5': [0.25],
'_PT_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt6': [0.25],
'_PT_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt7': [0.33],
'_CG_thd1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_CG_thd2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_AMT_trt1':[0],
'_AMT_trt2':[0],
'_NIIT_trt':[0],
'_STD': [[15000,30000,15000,15000,12600, 6300, 1050]],
'_II_em': [0],
'_ID_BenefitSurtax_crt': [0.0],
'_ID_BenefitSurtax_trt': [1.0],
'_ID_BenefitSurtax_em': [[100000, 200000, 100000, 100000, 200000, 100000]]
}}

reform_be = {2017: {'_BE_sub': [0.4],
                    '_BE_cg': [0]}}


In [4]:
policy_T.implement_reform(reform_T1)
policy_H.implement_reform(reform_T1)
calcbase.advance_to_year(2017)
calc_T.advance_to_year(2017)
calc_T_Be.advance_to_year(2017)

In [5]:
calcbase.calc_all()
calc_T.calc_all()
calc_T_Be.calc_all()
bhv.update_behavior(reform_be)

In [6]:
wage_mtr_x, wage_mtr_y = Behavior._mtr_xy(calcbase, calc_T_Be, mtr_of='e00200p', tax_type='combined')

In [7]:
PCH = ((1. - wage_mtr_y)/(1. - wage_mtr_x) - 1.)

In [10]:
delta_amt_org = PCH*0.55*calcbase.records.c04800

In [11]:
delta_amt_taxinc = (calc_behav.records.c04800 - calc_T.records.c04800)

In [12]:
delta_amt_iit = (calc_behav.records._iitax - calc_T.records._iitax)

In [27]:
df={}

In [28]:
df['RECID'] = calc_behav.records.RECID
df['delta_org'] = delta_amt_org
df['delta_taxinc'] = delta_amt_taxinc
df['delta_iit'] = delta_amt_iit

In [31]:
df = DataFrame(data=df)

In [33]:
df.to_csv("delta.csv")

In [6]:
(calcbase.records._combined * calcbase.records.s006).sum()/1000000000

2643.670402365125

In [7]:
(calc_behav.records._combined * calc_behav.records.s006).sum()/1000000000

2551.2135025364228

In [8]:
(calc_T.records._combined * calc_T.records.s006).sum()/1000000000

2508.0495247964759

In [6]:
create_diagnostic_table(calc_bbhe)

,2017
Returns (#m),172.4
AGI ($b),"11,245.6"
Itemizers (#m),20.8
Itemized Deduction ($b),930.3
Standard Deduction Filers (#m),138.3
Standard Deduction ($b),"2,778.0"
Personal Exemption ($b),0.0
Taxable Income ($b),"8,121.9"
Regular Tax ($b),"1,681.4"
AMT Income ($b),"10,723.0"


In [10]:
Trump_static = multiyear_diagnostic_table(calc_T, num_years=10)

In [11]:
Base = multiyear_diagnostic_table(calcbase, num_years=10)

In [8]:
Trump_BE = multiyear_diagnostic_table(calc_T_Be, num_years=10)

In [13]:
Base

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),172.4,174.1,175.6,177.1,178.4,179.8,181.0,182.3,183.6,184.8
AGI ($b),"10,960.8","11,406.5","11,828.8","12,284.2","12,786.1","13,327.2","13,900.3","14,505.9","15,142.5","15,815.1"
Itemizers (#m),45.4,46.0,46.5,47.1,47.5,48.1,48.6,49.1,49.6,50.2
Itemized Deduction ($b),"1,351.9","1,426.2","1,501.2","1,580.5","1,661.9","1,749.6","1,839.4","1,932.2","2,027.9","2,128.4"
Standard Deduction Filers (#m),113.6,114.4,115.1,115.8,116.3,116.8,117.2,117.7,118.2,118.7
Standard Deduction ($b),"1,023.1","1,055.6","1,087.4","1,121.4","1,155.5","1,191.3","1,225.9","1,262.6","1,301.0","1,341.5"
Personal Exemption ($b),"1,218.1","1,254.5","1,290.8","1,329.3","1,368.1","1,408.6","1,449.6","1,491.3","1,534.5","1,580.0"
Taxable Income ($b),"7,846.2","8,159.5","8,450.4","8,763.1","9,123.0","9,514.7","9,934.4","10,381.6","10,854.8","11,355.7"
Regular Tax ($b),"1,721.7","1,787.3","1,846.3","1,912.2","1,990.4","2,076.4","2,170.4","2,271.4","2,379.3","2,494.3"
AMT Income ($b),"10,233.6","10,636.8","11,016.4","11,426.5","11,881.4","12,372.6","12,894.5","13,446.7","14,027.7","14,641.7"


In [14]:
Trump_static

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),172.4,174.1,175.6,177.1,178.4,179.8,181.0,182.3,183.6,184.8
AGI ($b),"10,960.8","11,406.5","11,828.8","12,284.2","12,786.1","13,327.2","13,900.3","14,505.9","15,142.5","15,815.1"
Itemizers (#m),21.1,21.9,22.6,23.4,24.2,25.0,25.8,26.6,27.5,28.3
Itemized Deduction ($b),923.4,987.9,"1,052.6","1,122.6","1,196.6","1,274.5","1,354.9","1,437.2","1,524.7","1,617.8"
Standard Deduction Filers (#m),137.9,138.6,139.0,139.5,139.6,139.9,140.0,140.2,140.4,140.6
Standard Deduction ($b),"2,770.0","2,848.0","2,925.0","3,005.0","3,083.4","3,167.6","3,248.6","3,334.7","3,421.9","3,512.6"
Personal Exemption ($b),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taxable Income ($b),"7,854.2","8,172.0","8,467.4","8,785.1","9,149.9","9,546.5","9,971.8","10,424.3","10,903.0","11,408.8"
Regular Tax ($b),"1,607.3","1,670.4","1,727.5","1,790.6","1,865.1","1,946.7","2,035.9","2,131.4","2,233.4","2,341.8"
AMT Income ($b),"10,453.2","10,863.0","11,249.3","11,666.3","12,126.5","12,623.3","13,151.5","13,709.9","14,296.6","14,915.9"


In [9]:
Trump_BE

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),172.4,174.1,175.6,177.1,178.4,179.8,181.0,182.3,183.6,184.8
AGI ($b),"11,107.5","11,556.8","11,983.3","12,442.6","12,950.4","13,499.7","14,071.8","14,693.5","15,339.2","16,018.4"
Itemizers (#m),20.9,21.7,22.4,23.2,24.0,24.8,25.6,26.4,27.3,28.1
Itemized Deduction ($b),926.4,992.1,"1,056.6","1,126.7","1,200.6","1,278.6","1,360.0","1,443.4","1,530.7","1,623.0"
Standard Deduction Filers (#m),138.1,138.8,139.2,139.7,139.8,140.1,140.2,140.4,140.6,140.8
Standard Deduction ($b),"2,774.4","2,851.9","2,929.6","3,010.2","3,088.6","3,173.5","3,254.0","3,339.9","3,427.7","3,519.8"
Personal Exemption ($b),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taxable Income ($b),"7,992.4","8,313.0","8,612.0","8,933.0","9,303.7","9,707.5","10,138.7","10,599.1","11,086.3","11,599.6"
Regular Tax ($b),"1,645.3","1,709.0","1,766.8","1,830.7","1,906.7","1,990.1","2,081.0","2,178.7","2,282.9","2,393.4"
AMT Income ($b),"10,592.7","11,004.9","11,395.4","11,815.7","12,281.8","12,786.1","13,312.4","13,886.2","14,481.7","15,107.5"


In [12]:
Trump_static.iloc[18] - Base.iloc[18]

2017     -135.6
2018     -140.0
2019     -143.8
2020     -148.4
2021     -154.0
2022     -160.6
2023     -167.5
2024     -175.2
2025     -183.3
2026     -192.4
Name: Combined Liability ($b), dtype: float64

In [13]:
Trump_BE.iloc[18] - Base.iloc[18]

2017       15.3
2018       15.7
2019       17.3
2020       17.8
2021       18.7
2022       19.1
2023       19.1
2024       19.5
2025       19.5
2026       18.8
Name: Combined Liability ($b), dtype: float64

In [14]:
policy_base_pal = Policy()
records_base_pal = Records(puf)

policy_T_pal = Policy()
records_reform_pal = Records(puf)

policy_H_pal = Policy()
records_org_pal = Records(puf)
bhv_pal = Behavior()


calcbase_pal = Calculator(policy = policy_base_pal, records = records_base_pal)
calc_T_pal = Calculator(policy = policy_T_pal, records = records_reform_pal)
calc_T_Be_pal = Calculator(policy = policy_H_pal, records = records_org_pal,behavior = bhv_pal)



reform_T1_pal = {
2017: {'_II_rt1': [0.12],
'_II_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_II_rt2': [0.25],
'_II_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt3': [0.25],
'_II_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt4': [0.25],
'_II_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt5': [0.25],
'_II_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt6': [0.25],
'_II_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt7': [0.33],
'_PT_rt1': [0.12],
'_PT_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_PT_rt2': [0.25],
'_PT_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt3': [0.25],
'_PT_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt4': [0.25],
'_PT_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt5': [0.25],
'_PT_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt6': [0.25],
'_PT_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt7': [0.33],
'_CG_thd1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_CG_thd2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_AMT_trt1':[0],
'_AMT_trt2':[0],
'_NIIT_trt':[0],
'_STD': [[15000,30000,15000,15000,12600, 6300, 1050]],
'_II_em': [0],
'_ID_BenefitSurtax_crt': [0.0],
'_ID_BenefitSurtax_trt': [1.0],
'_ID_BenefitSurtax_em': [[100000, 200000, 100000, 100000, 200000, 100000]]
}}

reform_be_pal = {2017: {'_BE_sub': [0.55],
                    '_BE_cg': [-3.67]}}

policy_T_pal.implement_reform(reform_T1_pal)
policy_H_pal.implement_reform(reform_T1_pal)
Static = {}
Dynamic = {}

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [15]:
for i in range(10):
    YEAR = 2017 + i
    calcbase_pal.advance_to_year(YEAR)
    calc_T_pal.advance_to_year(YEAR)
    calc_T_Be_pal.advance_to_year(YEAR)
    calcbase_pal.calc_all()
    calc_T_pal.calc_all()
    calc_T_Be_pal.calc_all()
    bhv_pal.update_behavior(reform_be_pal)
    calc_behav_pal = Behavior.response(calcbase_pal, calc_T_Be_pal)
    calc_behav_pal.calc_all()
    Static_diif = proportional_change_gdp(calcbase_pal, calc_T_pal ,elasticity=0.36)
    Dynamic_diif = proportional_change_gdp(calcbase_pal, calc_behav_pal ,elasticity=0.36)
    Static[YEAR] = Static_diif
    Dynamic[YEAR] = Dynamic_diif

0.01093
0.00976
0.01077
0.00977
0.01071
0.00975
0.01056
0.00959
0.01058
0.00956
0.01064
0.00960
0.01055
0.00953
0.01055
0.00968
0.01064
0.00973
0.01062
0.00980


In [16]:
Static

{2017: 0.010931822535599536,
 2018: 0.010769699869210688,
 2019: 0.010711632792558497,
 2020: 0.010564684114397675,
 2021: 0.010579588699710834,
 2022: 0.010643047698061599,
 2023: 0.010551035050492652,
 2024: 0.010545129448755698,
 2025: 0.010644622031111986,
 2026: 0.010619342537506503}

In [17]:
Dynamic

{2017: 0.0097574739875893757,
 2018: 0.0097708092099408807,
 2019: 0.0097548736673857984,
 2020: 0.0095943689762045693,
 2021: 0.0095566577433498796,
 2022: 0.0096023036357736694,
 2023: 0.0095331039960088113,
 2024: 0.0096805956463961954,
 2025: 0.0097314399443700437,
 2026: 0.0098043737461806259}